In [1]:
!pip3 install gym_super_mario_bros==7.3.0 nes_py


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
#import the game
import gym_super_mario_bros
#import the Joypad Wrapper
from nes_py.wrappers import JoypadSpace
#import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [3]:
# Setup Game
env = gym_super_mario_bros.make('SuperMarioBros-v0',apply_api_compatibility=True,render_mode="human" )
env = JoypadSpace(env, SIMPLE_MOVEMENT)

/opt/homebrew/lib/python3.11/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(


In [ ]:
# Create a flag - restart or not
done = True
# Loop through each frame in the game
for step in range(100000):
    # Start the game to begin with
    if done:
        # Start the game
        env.reset()
        # Do random actions
    state, reward, done, _, info = env.step(env.action_space.sample())
    # Show the game on the screen
    env.render()
# Close the game
env.close()

In [ ]:
# Install pytorch
# pip3 install torch torchvision torchaudio
!pip3 install torch torchvision torchaudio

In [ ]:
# Install stable baselines for RL stuff
!pip3 install stable-baselines3\[extra\]

In [ ]:
# Import Grayscaling Wrapper
from gym.wrappers import GrayScaleObservation
# Import Vectorization Wrappers (stable baselines 3 Reinforcement Learning library - PPO-algorithm)
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import MatPlotLib to show the impact of FrameStacking
from matplotlib import pyplot as plt

In [5]:
# Something i found while googling, i think it has something to do with the seed: investigate further
JoypadSpace.reset = lambda self, **kwargs: self.env.reset(**kwargs)
# 1. Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0',apply_api_compatibility=True,render_mode="human" )
# 2. Simplify the controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. GrayScale the environment
env = GrayScaleObservation(env, keep_dim=True)
# 4. Wrap inside the dummy environment
env = DummyVecEnv([lambda: env])
# 5. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

NameError: name 'GrayScaleObservation' is not defined

In [ ]:
state = env.reset()

In [ ]:
state[0].shape

In [ ]:
state, reward, done, info = env.step([env.action_space.sample()])

In [ ]:
# Use MatPlotLib to show the game frame
plt.imshow(state[0])

In [ ]:
# Show 4 frames, it kind of gives our AI like a memory, so it does not just see one frame but actually 4.
# This way our AI knows that mario is jumping for example. (SIMPLE_MOVEMENT[5])
plt.figure(figsize=(20,16))
for idx in range(state.shape[3]):
    plt.subplot(1,4,idx+1)
    plt.imshow(state[0][:,:,idx])
plt.show()

In [ ]:
# import os for file path management
import os
# import PPO for algos
from stable_baselines3 import PPO
# import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
# import os for file path management
import os
# import PPO for algos
from stable_baselines3 import PPO
# import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
# setup model and saving callback
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)

In [ ]:
# This is the AI model started
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

In [ ]:
# Train the AI model, this is where the AI model starts to learn.
# If we do not want the callback, we can just remove it here so remove callback=callback
model.learn(total_timesteps=1000000, callback=callback)

In [ ]:
# Manually save the model
model.save('latest_model')

In [ ]:
# Load model
model = PPO.load('./train/best_model_900000')

In [ ]:
model.learn(total_timesteps=1000000, callback=callback)

In [ ]:
state = env.reset()

In [ ]:
SIMPLE_MOVEMENT[model.predict(state)[0][0]]

In [ ]:
# Start the game
state = env.reset()
# Loop through the game
while True:
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

In [ ]:
# Load model
model = PPO.load('./train/best_model_200000')

In [ ]:
model.set_env(env)

In [ ]:
model.learn(total_timesteps=1000000, callback=callback)